In [2]:
from transformers import pipeline
from transformers import AutoTokenizer
import pandas as pd
import jieba
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer 
#调用sklearn.preprocessing库的LabelEncoder方法对文章分类做标签编码
from sklearn.preprocessing import LabelEncoder
#调用sklearn.model_selection库的train_test_split方法划分训练集和测试集
import re

读入并清洗文本

定义函数，使得字符串中只保留汉字和句号

In [23]:
def is_chinese(uchar):
    if (uchar >= u'\u4e00' and uchar <= u'\u9fa5') or uchar==u'\u3002':  
    # 判断一个uchar是否是汉字,或者句号，用Unicode编码判断
        return True
    else:
        return False
 
def allcontents(contents):
    content = ''
    for i in contents:
        if is_chinese(i):
            content = content+i
    return content
    #print('\n处理后的句子为:\n'+content)

In [27]:
file_path = 'test.txt'
clear_txt = [] # 创建列表以存储读取文本的字符串列表

with open(file_path,'r') as f:
    content = f.readlines()
    for i in content:
        clear_txt.append(allcontents(i))
    #print(content)

# 在每个句子后添加换行符号 '\n'
for i in range(len(clear_txt)):
    clear_txt[i] = clear_txt[i].replace('。','。\n')

# 将字符串列表写入新文件
with open(file_path.replace('.txt','')+'_clear.txt','w') as f:
    f.writelines(clear_txt)

清洗词典

In [18]:
dictionary = ''
with open('dictionary.txt','r') as f:
    dictionary = f.read()
    
dictionary = dictionary.replace('、','\n')
dictionary = dictionary.replace('，','\n')
dictionary = dictionary.replace(' ','')

with open("dictionary_clear.txt", 'w') as f:
    f.write(dictionary)

计算包含词库中词的句子数量

In [38]:
text_name = file_path
dictionary = []
text = []
count = 0.0

with open(text_name, 'r') as f:
    text = [k.strip() for k in f.readlines()]

with open('dictionary_clear.txt', 'r') as f:
    dictionary = [k.strip() for k in f.readlines()]

# 删除词库中重复的词
dictionary = list(set(dictionary))
dictionary.pop(0)  # 字典最前方有个空字符串，要删掉，不然会对匹配结果造成影响
total_sentence = len(text)

for i in range(len(dictionary)):
    temp = []
    for j in range(len(text)):
        if dictionary[i] in text[j]:
            count += 1
            temp.append(j)
    temp.reverse()  # 对索引进行反转，从后往前删除，否则会出错
    for k in temp:
        del text[k] # 删除已经计算过的句子，以防该句子不同的词重复计算
           

In [34]:
print(dictionary)

['颗粒物', '水力发电设施', '毁灭性飓风', '灰池', '燃煤', '核电厂', '全球变暖', '化石发电', '冬季', '灾难天气', 'hcfcs,cfcs', '绿色', '灾害', '干燥天气', '废能源', '排放温室气体', '超洁净', '严冬', '大型太阳能', '洪涝灾害', '地震洪水', '创纪录的降雪', '水电', '余热', '破坏风暴', '发电厂', '粉末活化', '更严格的环境', '荒漠化', '常规发电', '电动机', '大飓风袭击', '碳中性', '废气', '氮氧化物控制', '有害空气污染物', '风能太阳能项目', '煤粉', 's02', '天然气', '由于大雨', '潮湿的天气状况', '较冷的天气', '严重干旱', '深度冻结', '凉爽的夏天', '异常天气', '龙卷风冰雹', '异常天气模式', '生物燃料', '风暴活动', '极端寒冷天气', '一氧化碳', '异常潮湿', '绿色能源', '泥石流', '热电联产设施', '热', '核电站', '氮氧化物燃烧器', '气候保护', '上岸', '降雨洪水', '飓风', '凉爽的春天', '融雪', '环境敏感区', '减少温室', '飓风地震', '潮湿天气', '影响飓风', '飓风登陆', '生物多样性', '洪水发生', '极端天气状况', '火山', '反常天气', '缺水', '异常寒冷', '岸风', '二氧化碳捕获', '反常寒冷', '温和正常', '煤炭燃烧', '碳氢化合物', '排放监测', '冰雹风暴', '严酷的冬季', '煤灰', '空气排放', '自然资源', '水排放', '大雪暴', '清洁燃烧', '反常', '化石燃料', '结合热电厂', '尾气排放', '飓风袭击', '火电', '硫氧化物', '燃气火电厂', '陆上风电场', '二氧化氮排放', '可持续发展', '可再生天然气', '温暖的冬天', '碳捕集', '地震飓风', '天气状况', '极端天气', '异常寒冷的天气', '水资源短缺', '降水', '减少污染', '异常恶劣天气', '飓风热带风暴', '温室气体', '太阳能太阳能', '较冷的正常', '冷温度', '凉爽天气', '浮游生物', '环境排放

In [35]:
print(dictionary)
print(text)
print(count)
print(total_sentence)
print(count/total_sentence)

['颗粒物', '水力发电设施', '毁灭性飓风', '灰池', '燃煤', '核电厂', '全球变暖', '化石发电', '冬季', '灾难天气', 'hcfcs,cfcs', '绿色', '灾害', '干燥天气', '废能源', '排放温室气体', '超洁净', '严冬', '大型太阳能', '洪涝灾害', '地震洪水', '创纪录的降雪', '水电', '余热', '破坏风暴', '发电厂', '粉末活化', '更严格的环境', '荒漠化', '常规发电', '电动机', '大飓风袭击', '碳中性', '废气', '氮氧化物控制', '有害空气污染物', '风能太阳能项目', '煤粉', 's02', '天然气', '由于大雨', '潮湿的天气状况', '较冷的天气', '严重干旱', '深度冻结', '凉爽的夏天', '异常天气', '龙卷风冰雹', '异常天气模式', '生物燃料', '风暴活动', '极端寒冷天气', '一氧化碳', '异常潮湿', '绿色能源', '泥石流', '热电联产设施', '热', '核电站', '氮氧化物燃烧器', '气候保护', '上岸', '降雨洪水', '飓风', '凉爽的春天', '融雪', '环境敏感区', '减少温室', '飓风地震', '潮湿天气', '影响飓风', '飓风登陆', '生物多样性', '洪水发生', '极端天气状况', '火山', '反常天气', '缺水', '异常寒冷', '岸风', '二氧化碳捕获', '反常寒冷', '温和正常', '煤炭燃烧', '碳氢化合物', '排放监测', '冰雹风暴', '严酷的冬季', '煤灰', '空气排放', '自然资源', '水排放', '大雪暴', '清洁燃烧', '反常', '化石燃料', '结合热电厂', '尾气排放', '飓风袭击', '火电', '硫氧化物', '燃气火电厂', '陆上风电场', '二氧化氮排放', '可持续发展', '可再生天然气', '温暖的冬天', '碳捕集', '地震飓风', '天气状况', '极端天气', '异常寒冷的天气', '水资源短缺', '降水', '减少污染', '异常恶劣天气', '飓风热带风暴', '温室气体', '太阳能太阳能', '较冷的正常', '冷温度', '凉爽天气', '浮游生物', '环境排放

In [37]:
print(dictionary)
print(text)
print(count)
print(total_sentence)
print(count/total_sentence)

['颗粒物', '水力发电设施', '毁灭性飓风', '灰池', '燃煤', '核电厂', '全球变暖', '化石发电', '冬季', '灾难天气', 'hcfcs,cfcs', '绿色', '灾害', '干燥天气', '废能源', '排放温室气体', '超洁净', '严冬', '大型太阳能', '洪涝灾害', '地震洪水', '创纪录的降雪', '水电', '余热', '破坏风暴', '发电厂', '粉末活化', '更严格的环境', '荒漠化', '常规发电', '电动机', '大飓风袭击', '碳中性', '废气', '氮氧化物控制', '有害空气污染物', '风能太阳能项目', '煤粉', 's02', '天然气', '由于大雨', '潮湿的天气状况', '较冷的天气', '严重干旱', '深度冻结', '凉爽的夏天', '异常天气', '龙卷风冰雹', '异常天气模式', '生物燃料', '风暴活动', '极端寒冷天气', '一氧化碳', '异常潮湿', '绿色能源', '泥石流', '热电联产设施', '热', '核电站', '氮氧化物燃烧器', '气候保护', '上岸', '降雨洪水', '飓风', '凉爽的春天', '融雪', '环境敏感区', '减少温室', '飓风地震', '潮湿天气', '影响飓风', '飓风登陆', '生物多样性', '洪水发生', '极端天气状况', '火山', '反常天气', '缺水', '异常寒冷', '岸风', '二氧化碳捕获', '反常寒冷', '温和正常', '煤炭燃烧', '碳氢化合物', '排放监测', '冰雹风暴', '严酷的冬季', '煤灰', '空气排放', '自然资源', '水排放', '大雪暴', '清洁燃烧', '反常', '化石燃料', '结合热电厂', '尾气排放', '飓风袭击', '火电', '硫氧化物', '燃气火电厂', '陆上风电场', '二氧化氮排放', '可持续发展', '可再生天然气', '温暖的冬天', '碳捕集', '地震飓风', '天气状况', '极端天气', '异常寒冷的天气', '水资源短缺', '降水', '减少污染', '异常恶劣天气', '飓风热带风暴', '温室气体', '太阳能太阳能', '较冷的正常', '冷温度', '凉爽天气', '浮游生物', '环境排放

In [39]:
print(dictionary)
print(text)
print(count)
print(total_sentence)
print(count/total_sentence)

['颗粒物', '水力发电设施', '毁灭性飓风', '灰池', '燃煤', '核电厂', '全球变暖', '化石发电', '冬季', '灾难天气', 'hcfcs,cfcs', '绿色', '灾害', '干燥天气', '废能源', '排放温室气体', '超洁净', '严冬', '大型太阳能', '洪涝灾害', '地震洪水', '创纪录的降雪', '水电', '余热', '破坏风暴', '发电厂', '粉末活化', '更严格的环境', '荒漠化', '常规发电', '电动机', '大飓风袭击', '碳中性', '废气', '氮氧化物控制', '有害空气污染物', '风能太阳能项目', '煤粉', 's02', '天然气', '由于大雨', '潮湿的天气状况', '较冷的天气', '严重干旱', '深度冻结', '凉爽的夏天', '异常天气', '龙卷风冰雹', '异常天气模式', '生物燃料', '风暴活动', '极端寒冷天气', '一氧化碳', '异常潮湿', '绿色能源', '泥石流', '热电联产设施', '热', '核电站', '氮氧化物燃烧器', '气候保护', '上岸', '降雨洪水', '飓风', '凉爽的春天', '融雪', '环境敏感区', '减少温室', '飓风地震', '潮湿天气', '影响飓风', '飓风登陆', '生物多样性', '洪水发生', '极端天气状况', '火山', '反常天气', '缺水', '异常寒冷', '岸风', '二氧化碳捕获', '反常寒冷', '温和正常', '煤炭燃烧', '碳氢化合物', '排放监测', '冰雹风暴', '严酷的冬季', '煤灰', '空气排放', '自然资源', '水排放', '大雪暴', '清洁燃烧', '反常', '化石燃料', '结合热电厂', '尾气排放', '飓风袭击', '火电', '硫氧化物', '燃气火电厂', '陆上风电场', '二氧化氮排放', '可持续发展', '可再生天然气', '温暖的冬天', '碳捕集', '地震飓风', '天气状况', '极端天气', '异常寒冷的天气', '水资源短缺', '降水', '减少污染', '异常恶劣天气', '飓风热带风暴', '温室气体', '太阳能太阳能', '较冷的正常', '冷温度', '凉爽天气', '浮游生物', '环境排放

# 一些功能函数

## 去除空格

In [1]:
contents = '   大家好， 欢迎一起来学习文本的空格   去除   ！'
print('处理前文本：'+contents)
def process(our_data):     #定义函数
    content = our_data.replace(' ','')   # 去掉文本中的空格
    print('处理后文本：'+content)
process(contents)

处理前文本：   大家好， 欢迎一起来学习文本的空格   去除   ！
处理后文本：大家好，欢迎一起来学习文本的空格去除！


## 去除空格的同时把！？省略号转换为句号

In [4]:
contents = '   大家好， 这里还有  很多的知识...一起拉学习吧 ！'
print('处理前文本：'+contents)
def process(data):     #定义函数
    content = data.replace(' ','')    # 去掉文本中的空格
    content = content.replace('...','。')    # 替换...
    content = content.replace('！','。')
    content = content.replace('？','。')
    print('处理后文本：'+content)
process(contents)

处理前文本：   大家好， 这里还有  很多的知识...一起拉学习吧 ！
处理后文本：大家好，这里还有很多的知识。一起拉学习吧。


## 让文本只保留汉字（和句号）

In [6]:
def is_chinese(uchar):
    if (uchar >= u'\u4e00' and uchar <= u'\u9fa5') or uchar==u'\u3002':  
    # 判断一个uchar是否是汉字,或者句号，用Unicode编码判断
        return True
    else:
        return False
 
def allcontents(contents):
    content = ''
    for i in contents:
        if is_chinese(i):
            content = content+i
    print('\n处理后的句子为:\n'+content)
 
centents = '1,2,3...我们开始吧， 加油。'
print('原句子为:\n'+centents)
allcontents(centents)

原句子为:
1,2,3...我们开始吧， 加油。

处理后的句子为:
我们开始吧加油。


## 文本中的表情符号去除

In [7]:
import re
sentence='现在听着音乐,duo rui mi,很开心*_*'
print('原句子为:\n'+sentence)
 
def clear_character(sentence):    
    pattern = re.compile("[^\u4e00-\u9fa5^,^.^!^a-z^A-Z^0-9]")  
    #只保留中英文、数字和符号，去掉其他东西
    #若只保留中英文和数字，则替换为[^\u4e00-\u9fa5^a-z^A-Z^0-9]
    line=re.sub(pattern,'',sentence)  #把文本中匹配到的字符替换成空字符
    new_sentence=''.join(line.split())    #去除空白
    print('\n处理后的句子为:\n'+new_sentence) 
 
clear_character(sentence)

原句子为:
现在听着音乐,duo rui mi,很开心*_*
现在听着音乐,duoruimi,很开心

处理后的句子为:
现在听着音乐,duoruimi,很开心


## 繁体中文与简体中文转换

In [ ]:
from opencc import OpenCC
 
sentence = '你现在读的这里是简体，這裡是繁體，能看懂嗎？'
print('原句子为:\n'+sentence)
 
 
def Simplified(sentence):
    new_sentence = Converter('zh-hans').convert(sentence)   # 繁体转为简体
    print('\n处理后的句子为:\n'+new_sentence) 
 
def Traditional(sentence):
    new_sentence = Converter('zh-hant').convert(sentence)   # 简体转为繁体
    print('\n处理后的句子为:\n'+new_sentence) 
 

Simplified(sentence)